In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
import os
import zipfile
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow as tf

In [0]:
def toxicity_prediction_CV(fold,validation_acc = None,validation_auc = None,validation_precision = None, validation_recall = None):
  local_zip = '//tmp/' + fold + '.zip'

  zip_ref = zipfile.ZipFile(local_zip, 'r')

  zip_ref.extractall('/tmp')
  zip_ref.close()

  # Define our example directories and files
  base_dir = '/tmp/' + fold

  pre_trained_model = VGG16(input_shape = (224, 224, 3), 
                                  include_top = False, 
                                  weights = 'imagenet')

  for layer in pre_trained_model.layers:
    layer.trainable = False

  # VGG16
  last_layer = pre_trained_model.get_layer('block5_pool')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output

  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(4096, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(4096, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)                  
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  #x = layers.Dense(2048, activation='relu')(x)
  # Add a dropout rate of 0.2
  #x = layers.Dropout(0.2)(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense(1, activation='sigmoid')(x)           

  model = Model( pre_trained_model.input, x) 

  for layer in pre_trained_model.layers:
    layer.trainable = False

  model.compile(optimizer = RMSprop(lr=0.0001), 
                loss = 'binary_crossentropy', 
                metrics = ['acc',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),
                          tf.keras.metrics.Recall()])

  train_dir = os.path.join( base_dir, 'train')
  validation_dir = os.path.join( base_dir, 'validation')


  train_nontoxicity_dir = os.path.join(train_dir, 'nontoxicity') # Directory with our training nontoxicity pictures
  train_toxicity_dir = os.path.join(train_dir, 'toxicity') # Directory with our training toxicity pictures
  validation_nontoxicity_dir = os.path.join(validation_dir, 'nontoxicity') # Directory with our validation nontoxicity pictures
  validation_toxicity_dir = os.path.join(validation_dir, 'toxicity')# Directory with our validation toxicity pictures

  train_nontoxicity_fnames = os.listdir(train_nontoxicity_dir)
  train_toxicity_fnames = os.listdir(train_toxicity_dir)

  from tensorflow.keras.preprocessing.image import ImageDataGenerator

  # Add our data-augmentation parameters to ImageDataGenerator
  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 40,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

  # Note that the validation data should not be augmented!
  test_datagen = ImageDataGenerator( rescale = 1.0/255. )

  # Flow training images in batches of 20 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(train_dir,
                                                      batch_size = 20,
                                                      class_mode = 'binary', 
                                                      target_size = (224, 224))     

  # Flow validation images in batches of 20 using test_datagen generator
  validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                            batch_size  = 1,
                                                            class_mode  = 'binary', 
                                                            target_size = (224, 224))

  classes_weight = {0:1.,
                    1:1.}
  
  class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc') > validation_acc):
          if(logs.get('val_auc') > validation_auc):
            if(logs.get('val_precision') > validation_precision):
              if(logs.get('val_recall') > validation_recall):
                print("\nReached early stopping criterion, accuracy for validation, so cancelling training!")
                self.model.stop_training = True

  callbacks = myCallback()

  model_history = model.fit_generator(
                    train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 20,
                    epochs = 200,
                    validation_steps = 7,
                    verbose = 1,
                    callbacks = [callbacks],
                    class_weight = classes_weight)
  return model_history

In [3]:
fold_one = toxicity_prediction_CV('fold one',validation_acc = 0.85,validation_auc = 0.9,validation_precision=0.5,validation_recall=0.5)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
20/20 [==============================] - 6s 304ms/step - loss: 1.7424 - acc: 0.5600 - auc: 0.5675 - precision: 0.5556 - recall: 0.5779 - val_loss: 0.9682 - val_acc: 0.5714 - val_auc: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200
20/20 [==============================] - 5s 262ms/step - loss: 0.7606 - acc: 0.5766 - auc: 0.6182 - precision: 0.5822 - recall: 0.6263 - val_loss: 1.2309 - val_acc: 0.4286 - val_auc: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
20/20 [==============================] - ETA: 0s - loss: 0.7832 - acc: 0.6182 - auc: 0.6420 - precision: 0.6311 - recall: 0.6468
Reached early stopping criterion, accuracy for validation, so cancelling training!
20/20 [==============================] - 5s 258ms/step - loss: 0.

In [0]:
Y_pred = model.predict_generator(validation_generator, 29)

In [3]:
toxicity_prediction_CV('fold two',validation_acc = 0.85,validation_auc = 0.7,validation_precision=0.5,validation_recall=0.5)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
20/20 [==============================] - 6s 302ms/step - loss: 1.8203 - acc: 0.5425 - auc: 0.5673 - precision: 0.5519 - recall: 0.5707 - val_loss: 1.1903 - val_acc: 0.5714 - val_auc: 0.5833 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200
20/20 [==============================] - 5s 262ms/step - loss: 0.7915 - acc: 0.5766 - auc: 0.6462 - precision: 0.5579 - recall: 0.5730 - val_loss: 1.7291 - val_acc: 0.4286 - val_auc: 0.1667 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 261ms/step - loss: 0.6411 - acc: 0.6571 - auc: 0.7082 - precision: 0.6914 - recall: 0.6080 - val_loss: 0.9262 - val_acc: 0.2857 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/200
20/20 [===========

KeyboardInterrupt: ignored

In [4]:
toxicity_prediction_CV('fold three',validation_acc = 0.99,validation_auc = 0.99,validation_precision=0.99,validation_recall=0.99)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/200
20/20 [==============================] - 6s 299ms/step - loss: 1.6298 - acc: 0.5350 - auc_1: 0.5699 - precision_1: 0.5507 - recall_1: 0.5507 - val_loss: 0.7239 - val_acc: 0.4286 - val_auc_1: 0.5000 - val_precision_1: 0.3333 - val_recall_1: 0.3333
Epoch 2/200
20/20 [==============================] - 5s 256ms/step - loss: 0.7895 - acc: 0.6156 - auc_1: 0.6453 - precision_1: 0.6114 - recall_1: 0.6178 - val_loss: 0.7543 - val_acc: 0.5714 - val_auc_1: 0.5000 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 257ms/step - loss: 0.7375 - acc: 0.5974 - auc_1: 0.6389 - precision_1: 0.6073 - recall_1: 0.5918 - val_loss: 0.6978 - val_acc: 0.4286 - val_auc_1: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 0.5000
Epoch 4/200
20/20 [==============================] - 5s 264ms/step - loss: 0.5902 - acc: 0

In [5]:
toxicity_prediction_CV('fold four',validation_acc = 0.99,validation_auc = 0.99,validation_precision=0.99,validation_recall=0.99)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/200
20/20 [==============================] - 6s 301ms/step - loss: 1.7805 - acc: 0.5450 - auc_2: 0.5401 - precision_2: 0.5663 - recall_2: 0.5337 - val_loss: 0.4937 - val_acc: 0.7143 - val_auc_2: 1.0000 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/200
20/20 [==============================] - 5s 266ms/step - loss: 0.8246 - acc: 0.5481 - auc_2: 0.5907 - precision_2: 0.5316 - recall_2: 0.5430 - val_loss: 0.6467 - val_acc: 0.5714 - val_auc_2: 0.8333 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 261ms/step - loss: 0.6903 - acc: 0.6286 - auc_2: 0.6761 - precision_2: 0.6373 - recall_2: 0.6533 - val_loss: 0.6700 - val_acc: 0.5714 - val_auc_2: 0.5000 - val_precision_2: 0.5000 - val_recall_2: 0.6667
Epoch 4/200
20/20 [==============================] - 5s 270ms/step - loss: 0.6734 

In [6]:
toxicity_prediction_CV('fold five',validation_acc = 0.99,validation_auc = 0.99,validation_precision=0.99,validation_recall=0.99)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/200
20/20 [==============================] - 6s 295ms/step - loss: 1.8383 - acc: 0.5175 - auc_3: 0.5028 - precision_3: 0.5150 - recall_3: 0.5176 - val_loss: 0.7321 - val_acc: 0.2857 - val_auc_3: 0.4167 - val_precision_3: 0.2500 - val_recall_3: 0.3333
Epoch 2/200
20/20 [==============================] - 5s 253ms/step - loss: 0.7133 - acc: 0.6052 - auc_3: 0.6471 - precision_3: 0.6133 - recall_3: 0.6798 - val_loss: 0.5984 - val_acc: 0.7143 - val_auc_3: 0.5000 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 251ms/step - loss: 0.6869 - acc: 0.6260 - auc_3: 0.7054 - precision_3: 0.6225 - recall_3: 0.6546 - val_loss: 0.6848 - val_acc: 0.5714 - val_auc_3: 0.6667 - val_precision_3: 0.6667 - val_recall_3: 0.5000
Epoch 4/200
20/20 [==============================] - 5s 260ms/step - loss: 0.6415 - acc: 0

TypeError: ignored

In [7]:
toxicity_prediction_CV('fold six',validation_acc = 0.99,validation_auc = 0.99,validation_precision=0.99,validation_recall=0.99)

last layer output shape:  (None, 7, 7, 512)
Found 545 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/200
20/20 [==============================] - 6s 291ms/step - loss: 1.8302 - acc: 0.5100 - auc_4: 0.5268 - precision_4: 0.5411 - recall_4: 0.5258 - val_loss: 0.7442 - val_acc: 0.5714 - val_auc_4: 0.8333 - val_precision_4: 0.5714 - val_recall_4: 1.0000
Epoch 2/200
20/20 [==============================] - 5s 251ms/step - loss: 0.7249 - acc: 0.6052 - auc_4: 0.6598 - precision_4: 0.5847 - recall_4: 0.5847 - val_loss: 0.9373 - val_acc: 0.7143 - val_auc_4: 1.0000 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 252ms/step - loss: 0.8254 - acc: 0.5688 - auc_4: 0.5839 - precision_4: 0.5777 - recall_4: 0.6010 - val_loss: 0.5309 - val_acc: 0.5714 - val_auc_4: 1.0000 - val_precision_4: 1.0000 - val_recall_4: 0.2500
Epoch 4/200
20/20 [==============================] - 5s 260ms/step - loss: 0.6873 - acc: 0

TypeError: ignored

In [8]:
toxicity_prediction_CV('fold seven',validation_acc = 0.99,validation_auc = 0.99,validation_precision=0.99,validation_recall=0.99)

last layer output shape:  (None, 7, 7, 512)
Found 546 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/200
20/20 [==============================] - 6s 292ms/step - loss: 1.5717 - acc: 0.5400 - auc_5: 0.5744 - precision_5: 0.5648 - recall_5: 0.5215 - val_loss: 0.8100 - val_acc: 0.4286 - val_auc_5: 0.5000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/200
20/20 [==============================] - 5s 263ms/step - loss: 0.7077 - acc: 0.6088 - auc_5: 0.6604 - precision_5: 0.5798 - recall_5: 0.6022 - val_loss: 0.8372 - val_acc: 0.2857 - val_auc_5: 0.5000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 3/200
20/20 [==============================] - 5s 252ms/step - loss: 0.6911 - acc: 0.6580 - auc_5: 0.7008 - precision_5: 0.6823 - recall_5: 0.6485 - val_loss: 0.8065 - val_acc: 0.7143 - val_auc_5: 0.5833 - val_precision_5: 0.6000 - val_recall_5: 1.0000
Epoch 4/200
20/20 [==============================] - 5s 259ms/step - loss: 0.6652 